In [1]:
from nltk import *
from collections import defaultdict
import string
import re,sys,pickle,math
import numpy as np
import pandas as pd
from nltk.corpus import wordnet
import heapq
import time

In [2]:
path_test = "test.csv"
test = pd.read_csv(path_test)
test.head()

,User_ID,Description,Browser_Used,Device_Used
0,id80132,Looking for a motel in close proximity to TV t...,Firefox,Mobile
1,id80133,Walking distance to Madison Square Garden and ...,InternetExplorer,Desktop
2,id80134,Visited Seattle on business. Spent - nights in...,IE,Tablet
3,id80135,This hotel location is excellent and the rooms...,Edge,Mobile
4,id80136,This hotel is awesome I love the service Antho...,Mozilla,Mobile


In [3]:
def word_net(word):
    """
    Aims at lemmatizing a word using synonyms of a word. 
    Longest prefix match of a word with some constraints is the lemmatized word.
    Synonyms of a word are extracted using synsets()
    """
    if(len(word)<=5):
        return word
    synonyms = []
    #antonyms = []
    for syn in wordnet.synsets(word):
        for l in syn.lemmas():
            synonyms.append(l.name())
            #if l.antonyms():
                #antonyms.append(l.antonyms()[0].name())
    best_match = ""
    match = 0
    if len(synonyms)==0:
        return word
    left_count = len(word)
    for syn in synonyms:
        if syn==word:
            continue
        temp = 0
        flag=1
        index = 0
        for i in range(min(len(syn),len(word))):
            index = i
            if(syn[i]!=word[i]):
                if((len(syn)-temp)<left_count):
                    left_count = len(syn)-temp
                    best_match = syn 
                    match = i
                flag=0
                break
            temp+=1
        if(flag==1):
            if((len(syn)-temp)<left_count):
                left_count = len(syn)-temp
                best_match = syn
                match = index+1
    if(len(word)-match>4 or len(best_match)>=len(word) or len(best_match)<=1):
        return word
    return best_match

def write_pkl(file,data):
    """
    Writes data from a pickle file. ie. "data.pkl"
    """
    writing = open(file, 'wb')
    pickle.dump(data, writing)
    return

def load_pkl(file):
    """
    Loads data from a pickle file. ie. "data.pkl"
    """
    pkl_file = open(file, 'rb')
    data = pickle.load(pkl_file)
    return data

corpus_size = 38932

def tf(word,doc_id):
    """
    Calculates and returns the term frequency log(tf+1)
    """
    try:
        return math.log(multimap_list[word][doc_id][1]+1)
    except:
        return 0

def idf(word):
    """
    Calculaltes and returns the inverse document frequency log(N/df)
    """
    try:
        return math.log(float(corpus_size)/len(multimap_list[word]))
    except:
        return 0

In [4]:
desc = np.array(test["Description"])
print(desc[1],desc.shape)

Walking distance to Madison Square Garden and the Penn Subway Station. Also very close to lots of retail stores and restaurants. The hotel offers free internet, a gym and a hot breakfast. Our room was on the top floor facing --th St. The room was clean and the staff very helpful and accommodating. The downside was that the A-C was making a noise and the faucet in the bathroom was loose - things that can be easily fixed. We walked from the hotel to Madison Square Gardens, to Times Square and on to Broadway. Everything considered, I will definitely recommend this hotel to my friends. (29404,)


In [5]:
"""
clean_test = []
with open("stp.txt") as file:
    stops = file.read().splitlines()
stops = set(stops)
for review in desc:
    review = re.sub(r'[^A-Za-z0-9\s]',r'',review)
    review = re.sub(r'\n',r' ',review)
    review = " ".join([w.lower() for w in review.split()])
    #review = " ".join([word_net(w) for w in review.split()])
    review = " ".join([word_net(w) for w in review.split() if w not in stops])
    clean_test.append(review)

write_pkl("test_data.pkl",clean_test)
"""
print("NOTE: This was carried out and stored in test_data.pkl")

NOTE: This was carried out and stored in test_data.pkl


In [6]:
#Loading the cleaned testing data
clean_test = load_pkl("test_data.pkl")
print(clean_test[1])
print(len(clean_test))

walk distance to madison square garden penn subway station also very close to lots of retail store restaurant hotel offer free internet gym hot breakfast room was on top floor face th st room was clean staff very helpful accommodate downside was ac was make noise faucet bath was loose thing can easy fixed walk hotel to madison square garden to times square on to broadway everything consider will definitely recommend hotel to friend
29404


In [7]:
#Loading the indexed multimap.pkl file
multimap_list = load_pkl("multimap.pkl")
print(multimap_list["drone"])

{30051: [[16], 1], 14951: [[152], 1], 35370: [[84], 1], 7409: [[176], 1], 30892: [[488], 1], 32305: [[45], 1], 24405: [[18], 1], 33847: [[51], 1], 13273: [[34], 1], 26939: [[24], 1], 27761: [[95], 1], 20287: [[65], 1]}


In [8]:
train_result = load_pkl("train_result.pkl")
print(train_result[:10])

[0 0 0 1 0 1 0 1 1 0]


In [9]:
pd.set_option('display.max_colwidth', -1)
test["Description"].head(1)

0    Looking for a motel in close proximity to TV taping of a Dr. Phil show, we chose the Dunes on Sunset Blvd in West Hollywood. Although the property displayed the AAA emblem, it certainly left a lot to be desired. There were chips & scrapes on the bottom of the door frame in the bathroom and the lotion containers were half full--apparently not replaced by housekeeping. We needed an early wakeup call, but couldn't use the clock radio alarm as there wasn't a radio in the room. There was no TV channel listing on the remote, or on the TV menu making viewing a chore.The TV remote had to be returned when checking-out. This place served its purpose, but not a place to revisit.
Name: Description, dtype: object

In [10]:
tfidf = load_pkl("tfidf.pkl")

In [11]:
def tf_idf(query):
    """
    Calculates tf_idf of a query and returns a list. ie. find[]
    """
    #positive = 0.0
    #negative = 0.0
    find = []
    for doc_id in range(corpus_size):
        value = 0.0
        for word in query.split(" "):
            try:
                value+= tfidf[word][doc_id+1][1]
            except:
                value+=0
        """if(train_result[doc_id]==1):
            positive+=value
        else:
            negative+=value"""
        if(value>0):
            heapq.heappush(find, (100000-value,doc_id+1,train_result[doc_id]))
    #total = (positive+negative)
    #return {positive/total,negative/total}
    return find

In [12]:
result = []
for review in clean_test[13:17]:
    answer = tf_idf(review)
    positive = 0.0
    negative = 0.0
    for ans in answer[:100]:
        if(ans[2]==1):
            positive+=(100000-ans[0])
        else:
            negative+=(100000-ans[0])
    total = (positive+negative)
    print(positive/total,negative/total)
    if(positive<negative):
        result.append(0)
    else:
        result.append(1)

0.39750839989473113 0.6024916001052688
0.4876278604312459 0.5123721395687542
0.5113455006728093 0.48865449932719074
0.537526624207721 0.462473375792279


In [13]:
clean_test[13:17]

['there reason many of previous review say place is awful is stay was one on only rooms available downtown due to convening within first few second of walk lobby notice dirty waslike hadnt vacuum weeks room was just bad overlook ally piled garbage crawl giant rats count least ten shabby depress roomnonsmoking reek of stale cigaret smoke water never got warm enough to take decent shower can go on but get pic',
 'great location close to times square many thing tried to find deal nearby night room was small used to nyc but was unclean smell moldy offer breakfast was minimal but of course better than nothing staff was not very friendly had social time than business time',
 'book hampton one night based on tripadvisor review usual were not disappoint have stay numerous hotel have come to love hampton inn consistent cleanliness value comfort one is near new lots of space comfy beds location isnt bad just off freeway maybe mile to lodo probably not walk distance of traffic freeway but very co

In [14]:
#dataframe = pd.DataFrame({'User_ID':test["User_ID"].tolist()[:4],'Is_Response':result})
#dataframe.to_csv('output1.csv')

In [15]:
tfidf["drone"]

{7409: [[176], 5.603862864030332],
 13273: [[34], 5.603862864030332],
 14951: [[152], 5.603862864030332],
 20287: [[65], 5.603862864030332],
 24405: [[18], 5.603862864030332],
 26939: [[24], 5.603862864030332],
 27761: [[95], 5.603862864030332],
 30051: [[16], 5.603862864030332],
 30892: [[488], 5.603862864030332],
 32305: [[45], 5.603862864030332],
 33847: [[51], 5.603862864030332],
 35370: [[84], 5.603862864030332]}

In [26]:
path_out1 = 'output1.csv'
out1 = pd.read_csv(path_out1)
print(out1.head(),out1.shape)

   Unnamed: 0 Is_Response  User_ID
0  0           not_happy   id80132
1  1           happy       id80133
2  2           not_happy   id80134
3  3           not_happy   id80135
4  4           happy       id80136 (19800, 3)


In [27]:
path_out2 = 'output2.csv'
out2 = pd.read_csv(path_out2)
print(out2.head(),out2.shape)

   Unnamed: 0 Is_Response  User_ID
0  0           not_happy   id99907
1  1           not_happy   id99908
2  2           happy       id99909
3  3           not_happy   id99910
4  4           happy       id99911 (9629, 3)


In [36]:
frames = [out1[:], out2[25:]]
result = pd.concat(frames)
print(result.shape,19800+9629)

(29404, 3) 29429


In [37]:
result.head(10)

,Unnamed: 0,Is_Response,User_ID
0,0,not_happy,id80132
1,1,happy,id80133
2,2,not_happy,id80134
3,3,not_happy,id80135
4,4,happy,id80136
5,5,happy,id80137
6,6,not_happy,id80138
7,7,not_happy,id80139
8,8,not_happy,id80140
9,9,happy,id80141
